# Sister City

Coursera course project to find sister/twin city for my home town using clustering.

## Introduction/Business Problem

With Britain living the EU, there is increased need for individuals in the EU to learn one of major European language author than English. In order to achieve this the plan is to move temporarily from my home City (Vilnius, Lithuania) to other EU and try to learn other language. 

I decided to use machine learning to find “sister” city to my home city.

Criteria for “sister’ city:
Countries to consider are Germany, France, Italy, Spain. The city of choice must be similar to my home city in Outdoors & Recreation facilities/landmarks offered. Also, the city has to be big enough (population over 400000). The plan is to live in central area of the city and only facilities/landmarks located near the city center need to be considered. 

## Data

1. List of European cities with population statistics published on  http://worldpopulationreview.com/continents/cities-in-europe/ will be used to find prospect cities. 

2. Foursquare data based on every city location will be used to get all Outdoors & Recreation facilities/landmarks.



